# 영화 리뷰 크롤링 (from Kinolights)
 1. 영화 목록 가져오기
 2. 리뷰 가져오기
 - Kinolights: 키노라이츠, OTT 통합검색 사이트

## 1. 영화 목록 가져오기
 - 리뷰를 긁어올 영화들의 목록 가져오기
 - 영화 - 판타지 - 평가 많은 순 정렬

In [1]:
import requests

In [2]:
url = 'https://api.kinolights.com/v1/search/discover?media_type=MOVIE&sort_filter=movie_rating_count_desc&exclude_watched=true&index_rating=0,100&rotten_score=0,100&imdb_score=1,10&star_rating=0,5&genres=11&nations=&years=1874,2023&show_time=1,200&offset=0&is_streaming=false'
# offset: 0이면 맨 처음 영화부터 가져옴
res = requests.get(url)
res.text    # json 형식의 응답 확인

'{"success":true,"results":[{"Idx":12318,"TitleKr":"센과 치히로의 행방불명","TitleEn":"The Spiriting Away Of Sen And Chihiro","MediaType":"MOVIE","HasTheater":0,"IndexRating":"GREEN","StarRating":4.26,"IndexScore":98.27,"Me_WatchedAt":null,"Me_IndexRating":null,"Me_StarRating":0,"Me_WatchList":0,"PosterImageUrl":"https:\\/\\/nujhrcqkiwag1408085.cdn.ntruss.com\\/static\\/upload\\/movie_poster_images\\/280x400\\/movie_12318_1531793963.jpg"},{"Idx":45823,"TitleKr":"트루먼 쇼","TitleEn":"The Truman Show","MediaType":"MOVIE","HasTheater":0,"IndexRating":"GREEN","StarRating":4.22,"IndexScore":99.02,"Me_WatchedAt":null,"Me_IndexRating":null,"Me_StarRating":0,"Me_WatchList":0,"PosterImageUrl":"https:\\/\\/nujhrcqkiwag1408085.cdn.ntruss.com\\/static\\/upload\\/movie_poster_images\\/280x400\\/movie_45823_1542678716.jpg"},{"Idx":62415,"TitleKr":"어벤져스: 인피니티 워","TitleEn":"Avengers: Infinity War","MediaType":"MOVIE","HasTheater":0,"IndexRating":"GREEN","StarRating":3.88,"IndexScore":95.61,"Me_WatchedAt":null,"Me_

In [3]:
# json 형식의 응답을 읽기 위해 json 라이브러리 사용
import json

j = json.loads(res.text)
j

{'results': [{'HasTheater': 0,
   'Idx': 12318,
   'IndexRating': 'GREEN',
   'IndexScore': 98.27,
   'Me_IndexRating': None,
   'Me_StarRating': 0,
   'Me_WatchList': 0,
   'Me_WatchedAt': None,
   'MediaType': 'MOVIE',
   'PosterImageUrl': 'https://nujhrcqkiwag1408085.cdn.ntruss.com/static/upload/movie_poster_images/280x400/movie_12318_1531793963.jpg',
   'StarRating': 4.26,
   'TitleEn': 'The Spiriting Away Of Sen And Chihiro',
   'TitleKr': '센과 치히로의 행방불명'},
  {'HasTheater': 0,
   'Idx': 45823,
   'IndexRating': 'GREEN',
   'IndexScore': 99.02,
   'Me_IndexRating': None,
   'Me_StarRating': 0,
   'Me_WatchList': 0,
   'Me_WatchedAt': None,
   'MediaType': 'MOVIE',
   'PosterImageUrl': 'https://nujhrcqkiwag1408085.cdn.ntruss.com/static/upload/movie_poster_images/280x400/movie_45823_1542678716.jpg',
   'StarRating': 4.22,
   'TitleEn': 'The Truman Show',
   'TitleKr': '트루먼 쇼'},
  {'HasTheater': 0,
   'Idx': 62415,
   'IndexRating': 'GREEN',
   'IndexScore': 95.61,
   'Me_IndexRating':

In [4]:
# 영화 목록 생성 (영화 인덱스, 영화 제목) 형식
movie_list = []

for movie in j['results']:
    movie_list.append((movie['Idx'], movie['TitleKr']))
print(len(movie_list))
movie_list[:5]

40


[(12318, '센과 치히로의 행방불명'),
 (45823, '트루먼 쇼'),
 (62415, '어벤져스: 인피니티 워'),
 (12183, '겨울왕국'),
 (12311, '하울의 움직이는 성')]

## 2. 리뷰 가져오기

In [15]:
# review_url = 'https://api.kinolights.com/v1/movie/47827/reviews?sort_filter=review_like_desc&offset=20&limit=10'
reviews = []
for movie in movie_list:
    for offset in range(0, 20):
        movie_idx = movie[0]
        review_url = f'https://api.kinolights.com/v1/movie/{movie_idx}/reviews?sort_filter=review_like_desc&offset={offset*10}&limit=10'
        request = requests.get(review_url)
        json_load = json.loads(request.text)
        data = json_load['data']
        for i in data:
            review_title = i['ReviewTitle'] if i['ReviewTitle'] else ''
            review = i['Review'] if i['Review'] else ''
            if review_title or review:
                reviews.append(review_title + review)

reviews[:5]

['어릴때 수없이 봤지만 처음 보는 영화처럼 봤다. “한 번 알게된 것은 절대 잊지 않는다, 생각이 나지 않을 뿐이지.” 돌아갈 곳이 있다는 것과 내가 누군지 안다는 것이 이렇게나 중요하며, 그 두가지의 부재가 이렇게도 슬플 수 있구나.\n+) 비하인드 찾다가 발견. 하쿠 성우가 자라서 하이큐 스가...ㄷㄷ',
 '\'너의 이름은\'보다 강한 메시지미야자키 하야오가 만든 애니메이션들의 특징은, 하나같이 동화를 배경으로 그 동화에 우리의 인생관을 녹여내 \'어른동화\'로 불린다. 그중 최고 역작이자, 아카데미와 베를린을 휩쓸고 BBC 죽기 전 봐야할 영화 상위권에 랭크된 \'센과 치히로의 행방불명\'은 단연 최고다.\n\n\'센과 치히로의 행방불명\'이 탄생하게 된 계기는 영화의 주인공인 \'치히로\'의 나이 또래인 미야자키 하야오의 친구 딸 때문이었다. 그는 "어린이는 여러 가지 체험을 하며 어른으로 성장한다. 그런 과정이 지나지 않으면 그 다음은 오지 않는다."는 메시지를 전달하기 위해 만들었다고 밝혔다.\n\t\n그러나 \'센과 치히로의 행방불명\'이 단순히 치히로의 성장만을 담은 전체관람가용 애니메이션으로 보면 곤란하다. \'센과 치히로의 행방불명\'은 현대 사회를 향한 강력한 메시지가 담겨 있다.\n\n마녀 \'유바바\'가 운영하는 온천여관은 오늘날 현대 사회를 압축시켜 놓은 공간이다. \'가마 할아범\'을 비롯한 직원들은 온천 운영을 위해 제대로 된 휴식조차 취하지 못한 채 치열하게 일하지만, 그들에게 돌아가는 합당한 대가나 보상은 없다.\n\n얻은 이익은 오로지 유바바 한 명의 부 축적으로 이어져 그녀의 방은 금붙이와 화려한 장식품들로 가득 차지만, 직원들은 노동에 대한 인식이 무뎌지고 있었다. \'가오나시\'가 여관직원들로부터 관심을 얻기 위해 사금을 꺼내 들었을 때, 여관직원들이 득달같이 달려든 것도 이 때문이다.\n\n돈의 맛에 빠진 유바바는 치히로와 노동 계약서를 작성할 때, 그녀의 이름보다도 치히로가 자신의 부를 축적하는 데 도움이 될 수 

In [16]:
len(reviews)

2505

In [17]:
reviews

['어릴때 수없이 봤지만 처음 보는 영화처럼 봤다. “한 번 알게된 것은 절대 잊지 않는다, 생각이 나지 않을 뿐이지.” 돌아갈 곳이 있다는 것과 내가 누군지 안다는 것이 이렇게나 중요하며, 그 두가지의 부재가 이렇게도 슬플 수 있구나.\n+) 비하인드 찾다가 발견. 하쿠 성우가 자라서 하이큐 스가...ㄷㄷ',
 '\'너의 이름은\'보다 강한 메시지미야자키 하야오가 만든 애니메이션들의 특징은, 하나같이 동화를 배경으로 그 동화에 우리의 인생관을 녹여내 \'어른동화\'로 불린다. 그중 최고 역작이자, 아카데미와 베를린을 휩쓸고 BBC 죽기 전 봐야할 영화 상위권에 랭크된 \'센과 치히로의 행방불명\'은 단연 최고다.\n\n\'센과 치히로의 행방불명\'이 탄생하게 된 계기는 영화의 주인공인 \'치히로\'의 나이 또래인 미야자키 하야오의 친구 딸 때문이었다. 그는 "어린이는 여러 가지 체험을 하며 어른으로 성장한다. 그런 과정이 지나지 않으면 그 다음은 오지 않는다."는 메시지를 전달하기 위해 만들었다고 밝혔다.\n\t\n그러나 \'센과 치히로의 행방불명\'이 단순히 치히로의 성장만을 담은 전체관람가용 애니메이션으로 보면 곤란하다. \'센과 치히로의 행방불명\'은 현대 사회를 향한 강력한 메시지가 담겨 있다.\n\n마녀 \'유바바\'가 운영하는 온천여관은 오늘날 현대 사회를 압축시켜 놓은 공간이다. \'가마 할아범\'을 비롯한 직원들은 온천 운영을 위해 제대로 된 휴식조차 취하지 못한 채 치열하게 일하지만, 그들에게 돌아가는 합당한 대가나 보상은 없다.\n\n얻은 이익은 오로지 유바바 한 명의 부 축적으로 이어져 그녀의 방은 금붙이와 화려한 장식품들로 가득 차지만, 직원들은 노동에 대한 인식이 무뎌지고 있었다. \'가오나시\'가 여관직원들로부터 관심을 얻기 위해 사금을 꺼내 들었을 때, 여관직원들이 득달같이 달려든 것도 이 때문이다.\n\n돈의 맛에 빠진 유바바는 치히로와 노동 계약서를 작성할 때, 그녀의 이름보다도 치히로가 자신의 부를 축적하는 데 도움이 될 수 